# Projeto Final | Sistema de Controle Financeiro

%----------------------------------------------------------------------------%<br>
<br>
``Autores:`` Andrea Elias, Anthony Heimlich, Éverton Donato, Julia Midori e Luana Kruger  <br>
<br>
``Instituição:`` ADA Tech<br>
<br>
``Projeto:`` Santander Coders 2023.2<br>
<br>
``Descrição:`` Este código desenvolve um sistema para controle financeiro que receba as movimentações e as armazena em um arquivo csv ou json.<br>
<br>
``Repositório GitHub:`` https://github.com/JuliaMidoriRW/Trabalho_Final_LogProgII_Grupo4 <br>
<br>

%----------------------------------------------------------------------------%

# Carregar bibliotecas e lista de registros

In [28]:
from datetime import datetime
from dateutil import relativedelta
import os

In [29]:
registros = []

# Carregar funções

### Função Calcular rendimento

In [3]:
def calcula_rendimento(valor, data):
    """
    Calcular montante com base no valor e na data do registro.

    Parameters
    ----------
    valor: float
        Valor do registro financeiro.
        
    data: datetime
        Data do registro no formato 'YYYY-MM-DD'.

    Returns
    -------
    montante: float
        Valor do rendimento considerando uma taxa de rendimento diário.

    """
    
    data_registro = datetime.strptime(data, "%Y-%m-%d").date()
    data_agora = datetime.now().date()
    delta = relativedelta.relativedelta(data_agora, data_registro)

    meses = delta.months + (delta.years * 12)
    taxa = 0.01  # Exemplo: taxa de rendimento diária (1%)
    montante = (valor * ((1 + taxa) ** meses)) - valor

    return montante

### Função Criar registro

In [4]:
def criar_registro(tipo, valor, data):
    """
    Criar novos registros e identificar a data que o registro foi feito, qual tipo de movimentação, valor.

    Parameters
    ----------
    tipo: string
        Tipo da movimentação realizada. Os tipos podem ser:
        - Receita (valor numérico e armazenado normalmente), 
        - Despesas (valor positivo, mas armazenado como negativo),
        - Investimento (tem a informação de 'Montante').

    valor: float
        Valor do registro.

    data: datetime
        Data do registro no formato 'YYYY-MM-DD'.

    Returns
    -------
    None

    Exemplo
    -------
    >>> criar_registro("Receita", 100, "2022-01-01")
    >>> criar_registro("Despesa", 50, "2022-01-02")
    >>> criar_registro("Investimento", 200, "2022-01-03")

    """

    data_registro = datetime.strptime(data, "%Y-%m-%d").date()
    dia, mes, ano = data_registro.day, data_registro.month, data_registro.year
    montante = 0

    if tipo == 'Receita':
        valor = float(valor)
    elif tipo == 'Despesa':
        valor = -float(valor)
    elif tipo == 'Investimento':
        valor = float(valor)
        montante = calcula_rendimento(valor, data)
    else:
        raise ValueError("Tipo de movimentação inválida.")
    
    registro = {'tipo': tipo, 'valor': valor, 'dia': dia, 'mes': mes, 'ano': ano}

    if montante > 0:
        registro['montante'] = montante
        
    registros.append(registro)

### Função Ler registro

In [5]:
def ler_registros(chave, valor):
    """
    Consultar os registros por data, tipo ou valor.

    Parameters
    ----------
    chave: string
        Chave pela qual os registros serão consultados ('data', 'tipo' ou 'valor').

    valor: string or float
        Valor associado à chave para filtrar os registros.

    Returns
    -------
    resultado: list[dict]
        Lista de registros que correspondem aos critérios de consulta.


    Exemplo
    -------
    >>> ler_registros('tipo', 'Receita')
    >>> ler_registros('data', '2022-02-15')
    >>> ler_registros('valor', 1000)

    """

    resultado = []

    for movimento in registros:
        if chave != "data":
            if movimento[chave] == valor:
                resultado.append(movimento)
        else:
            data_registro = datetime.strptime(valor, "%Y-%m-%d").date()
            dia, mes, ano = data_registro.day, data_registro.month, data_registro.year
            if movimento["dia"] == dia and movimento["mes"] == mes and movimento["ano"] == ano:
                resultado.append(movimento)

    return resultado


### Função Atualizar registro

In [6]:
def atualizar_registro(indice, tipo, valor):
    """
    Atualiza o valor e/ou o tipo do registro no índice especificado.

    Parameters
    ----------
    indice: int
        Índice do registro a ser atualizado.

    tipo: string
        Tipo da movimentação financeira ('Receita', 'Despesa' ou 'Investimento').

    valor: float
        Valor da movimentação financeira.

    Returns
    -------
    None

    Nota
    -------
    - Se o índice estiver fora dos limites dos registros existentes, nenhum registro será atualizado.
    - A data do registro é automaticamente atualizada para a data e hora atuais.
    - Se o tipo for especificado como 'Investimento', o montante também será recalculado.

    Exemplo
    -------
    >>> atualizar_registro(0, tipo='Receita', valor=1500.0)
    >>> atualizar_registro(2, tipo='Investimento', valor=100)
    
    """
    
    if indice < len(registros):
        registro = registros[indice]
        if tipo:
            registro['tipo'] = tipo
        if valor:
            registro['valor'] = valor
            
        data_agora = datetime.now()
        dia, mes, ano = data_agora.day, data_agora.month, data_agora.year
        registro['dia'], registro['mes'], registro['ano'] = dia, mes, ano

        if registro['tipo'] == "Investimento":
            registro['montante'] = calcula_rendimento(registro['valor'], f"{ano}-{mes:02d}-{dia:02d}")

        print('Registro', indice, 'atualizado com sucesso')
    
    else:
        print('Índice fora dos limites dos registros existentes, nenhum registro foi atualizado.')

### Função Deletar registro

In [7]:
def deletar_registro(indice):
    """
    Deleta o registro no índice especificado.

    Parameters
    ----------
    indice: int
        Indice do registro a ser deletado.

    Returns
    -------
    None

    Nota
    -------
    - Se o índice estiver fora dos limites dos registros existentes, nenhum registro será deletado.

    Exemplo
    -------
    >>> deletar_registro(0)
    >>> deletar_registro(2)

    """
    if indice < len(registros):
        del registros[indice]
        print('Registro', indice, 'deletado com sucesso')
    
    else:
        print('Índice fora dos limites dos registros existentes, nenhum registro foi deletado.')

### Função Atualizar rendimento

In [8]:
def atualiza_rendimento():
    """
    Atualiza os valores de rendimento para todas as movimentações do tipo 'Investimento'.

    Parameters
    ----------
    None

    Returns
    -------
    None

    Nota
    -------
    - Atualiza os montantes para todas as movimentações do tipo 'Investimento' com base na taxa de rendimento diária.
    - A data do investimento é obtida a partir dos campos 'dia', 'mes' e 'ano' de cada movimentação.

    Exemplo
    -------
    >>> atualiza_rendimento()

    """
    
    for movimento in registros:
        if movimento["tipo"] == "Investimento":
            valor_inicial = movimento["valor"]
            concatenar = str(movimento['ano']) + '-' + str(movimento['mes']) + '-' + str(movimento['dia'])
            data_investimento = datetime.strptime(concatenar, "%Y-%m-%d")
            movimento["montante"] = calcula_rendimento(valor_inicial, data_investimento.strftime("%Y-%m-%d"))

### Função Exportar relatório

In [9]:
def exportar_relatorio():
    """
    Exporta um relatório final em formato CSV.

    Parameters
    ----------
    None

    Returns
    -------
    None

    Nota
    -------
    - O relatório é exportado para um arquivo com nome padrão 'relatorio.csv'.
    - O relatório inclui informações sobre tipo, valor, data e montante para cada movimentação no formato especificado.

    Exemplo:
    >>> exportar_relatorio()

    """

    nome_arquivo = "relatorio.csv"

    with open(nome_arquivo, "w") as file:
        file.write("Tipo, Valor, Data, Montante\n")
        for movimento in registros:
            tipo = movimento.get('tipo', '')
            valor = movimento.get('valor', '')
            dia = movimento.get('dia', '')
            mes = movimento.get('mes', '')
            ano = movimento.get('ano', '')
            montante = movimento.get('montante', 0)


            data_formatada = f"{ano}-{mes:02d}-{dia:02d}"

            file.write(f"{tipo},{valor},{data_formatada},{montante}\n")

    print(f"Relatório exportado para {nome_arquivo}")


### Função Agrupar 

In [10]:
def agrupar_por(chave):
    """
    Função de agrupamento capaz de mostrar o total de valor baseado em alguma informação (tipo, mes, ano, data).

    Parameters
    ----------
    chave: string
        Informação base para o agrupamento ('tipo', 'mes', 'ano', 'data')

    Returns
    -------
    resultado: dict
        Dicionário com o registro agrupado por chave

    """
        
    resultado = {}
    for movimento in registros:
        valor = movimento["valor"]
        if chave == "tipo":
            chave_valor = movimento["tipo"]
        elif chave == "mes":
            chave_valor = movimento["mes"]
        elif chave == "ano":
            chave_valor = movimento["ano"]
        else:
            chave_valor = movimento["dia"]

        resultado[chave_valor] = resultado.get(chave_valor, 0) + valor

    return resultado

# Testes executados

### Criar registro

In [11]:
criar_registro("Receita", 100, "2022-01-01")
criar_registro("Despesa", 50, "2022-01-02")
criar_registro("Investimento", 200, "2022-01-03")
criar_registro("Receita", 1000, "2022-01-01")
criar_registro("Despesa", 90, "2022-01-02")
criar_registro("Investimento", 200, "2023-05-03")
criar_registro("Receita", 550, "2023-02-03")

registros

[{'tipo': 'Receita', 'valor': 100.0, 'dia': 1, 'mes': 1, 'ano': 2022},
 {'tipo': 'Despesa', 'valor': -50.0, 'dia': 2, 'mes': 1, 'ano': 2022},
 {'tipo': 'Investimento',
  'valor': 200.0,
  'dia': 3,
  'mes': 1,
  'ano': 2022,
  'montante': 53.946929706382946},
 {'tipo': 'Receita', 'valor': 1000.0, 'dia': 1, 'mes': 1, 'ano': 2022},
 {'tipo': 'Despesa', 'valor': -90.0, 'dia': 2, 'mes': 1, 'ano': 2022},
 {'tipo': 'Investimento',
  'valor': 200.0,
  'dia': 3,
  'mes': 5,
  'ano': 2023,
  'montante': 16.571341125616044},
 {'tipo': 'Receita', 'valor': 550.0, 'dia': 3, 'mes': 2, 'ano': 2023}]

### Ler registro

In [12]:
print(ler_registros('tipo', 'Receita'))

print(ler_registros('data', '2022-01-03'))

print(ler_registros('valor', 1000))

[{'tipo': 'Receita', 'valor': 100.0, 'dia': 1, 'mes': 1, 'ano': 2022}, {'tipo': 'Receita', 'valor': 1000.0, 'dia': 1, 'mes': 1, 'ano': 2022}, {'tipo': 'Receita', 'valor': 550.0, 'dia': 3, 'mes': 2, 'ano': 2023}]
[{'tipo': 'Investimento', 'valor': 200.0, 'dia': 3, 'mes': 1, 'ano': 2022, 'montante': 53.946929706382946}]
[{'tipo': 'Receita', 'valor': 1000.0, 'dia': 1, 'mes': 1, 'ano': 2022}]


### Atualizar registro

In [13]:
print(registros[0])
atualizar_registro(0, tipo='Receita', valor=1500.0)
print(registros[0])

{'tipo': 'Receita', 'valor': 100.0, 'dia': 1, 'mes': 1, 'ano': 2022}
Registro 0 atualizado com sucesso
{'tipo': 'Receita', 'valor': 1500.0, 'dia': 22, 'mes': 1, 'ano': 2024}


In [15]:
# Verificar a atualização do montante

print(registros[2])
atualizar_registro(2, tipo='Investimento', valor=100)
print(registros[2])

{'tipo': 'Investimento', 'valor': 100, 'dia': 22, 'mes': 1, 'ano': 2024, 'montante': 0.0}
Registro 2 atualizado com sucesso
{'tipo': 'Investimento', 'valor': 100, 'dia': 22, 'mes': 1, 'ano': 2024, 'montante': 0.0}


### Deletar registro

In [16]:
registros[1]

{'tipo': 'Despesa', 'valor': -50.0, 'dia': 2, 'mes': 1, 'ano': 2022}

In [17]:
deletar_registro(1)

Registro 1 deletado com sucesso


In [18]:
registros[1]

{'tipo': 'Investimento',
 'valor': 100,
 'dia': 22,
 'mes': 1,
 'ano': 2024,
 'montante': 0.0}

### Atualizar rendimento

In [19]:
# investimentos = [registro for registro in registros if registro['tipo'] == 'Investimento']
print(ler_registros('tipo', 'Investimento'))

[{'tipo': 'Investimento', 'valor': 100, 'dia': 22, 'mes': 1, 'ano': 2024, 'montante': 0.0}, {'tipo': 'Investimento', 'valor': 200.0, 'dia': 3, 'mes': 5, 'ano': 2023, 'montante': 16.571341125616044}]


In [20]:
atualiza_rendimento()

In [21]:
print(ler_registros('tipo', 'Investimento'))

[{'tipo': 'Investimento', 'valor': 100, 'dia': 22, 'mes': 1, 'ano': 2024, 'montante': 0.0}, {'tipo': 'Investimento', 'valor': 200.0, 'dia': 3, 'mes': 5, 'ano': 2023, 'montante': 16.571341125616044}]


### Exportar relatorio .CSV

In [22]:
exportar_relatorio()

Relatório exportado para relatorio.csv


### Agrupar por tipo

In [23]:
print("Agrupado por tipo:", agrupar_por("tipo"))

## Não funciona com mês, ano e data
print("Agrupado por mês:", agrupar_por("mes"))
print("Agrupado por ano:", agrupar_por("ano"))
print("Agrupado por data:", agrupar_por("data"))

Agrupado por tipo: {'Receita': 3050.0, 'Investimento': 300.0, 'Despesa': -90.0}
Agrupado por mês: {1: 2510.0, 5: 200.0, 2: 550.0}
Agrupado por ano: {2024: 1600.0, 2022: 910.0, 2023: 750.0}
Agrupado por data: {22: 1600.0, 1: 1000.0, 2: -90.0, 3: 750.0}


### Consultas por tipo

In [24]:
print("Pesquisa por tipo Receita:", ler_registros("tipo",'Receita'))
print("Pesquisa por tipo Despesa:", ler_registros("tipo",'Despesa'))
print("Pesquisa por tipo Investimento:", ler_registros("tipo",'Investimento'))

print("Pesquisa por valor:", ler_registros("valor",100))
print("Pesquisa por valor:", ler_registros("valor",-50))

print("Pesquisa por data:", ler_registros("data","2022-01-01"))
print("Pesquisa por data:", ler_registros("data","2023-05-03"))

Pesquisa por tipo Receita: [{'tipo': 'Receita', 'valor': 1500.0, 'dia': 22, 'mes': 1, 'ano': 2024}, {'tipo': 'Receita', 'valor': 1000.0, 'dia': 1, 'mes': 1, 'ano': 2022}, {'tipo': 'Receita', 'valor': 550.0, 'dia': 3, 'mes': 2, 'ano': 2023}]
Pesquisa por tipo Despesa: [{'tipo': 'Despesa', 'valor': -90.0, 'dia': 2, 'mes': 1, 'ano': 2022}]
Pesquisa por tipo Investimento: [{'tipo': 'Investimento', 'valor': 100, 'dia': 22, 'mes': 1, 'ano': 2024, 'montante': 0.0}, {'tipo': 'Investimento', 'valor': 200.0, 'dia': 3, 'mes': 5, 'ano': 2023, 'montante': 16.571341125616044}]
Pesquisa por valor: [{'tipo': 'Investimento', 'valor': 100, 'dia': 22, 'mes': 1, 'ano': 2024, 'montante': 0.0}]
Pesquisa por valor: []
Pesquisa por data: [{'tipo': 'Receita', 'valor': 1000.0, 'dia': 1, 'mes': 1, 'ano': 2022}]
Pesquisa por data: [{'tipo': 'Investimento', 'valor': 200.0, 'dia': 3, 'mes': 5, 'ano': 2023, 'montante': 16.571341125616044}]


# Testes com interação com o usuário

In [30]:
while True:
    print("\n----- Menu -----")
    print("1. Criar Registro")
    print("2. Ler Registros")
    print("3. Atualizar Registro")
    print("4. Deletar Registro")
    print("5. Atualizar Rendimento")
    print("6. Exportar Relatório")
    print("7. Agrupar Registros")
    print("8. Sair")

    escolha = input("Escolha uma opção (1-8): ")

    if escolha == '1':
        while True:
            tipo = input("Tipo (Receita/Despesa/Investimento): ").capitalize()
            if tipo in ['Receita', 'Despesa', 'Investimento']:
                break
            else:
                print("Tipo inválido! Insira um tipo válido")

        valor = float(input("Valor: "))
        data = input("Data (YYYY-MM-DD): ")
        criar_registro(tipo, valor, data)
        print("Registro criado com sucesso!")

    elif escolha == '2':
        while True:
            chave = input("Chave para consulta (tipo/valor/data): ").lower()
            if chave in ['tipo','valor','data']:
                break
            else:
                print("Chave inválida! Insira uma chave válida")

        valor = input("Valor para consulta: ")
        resultado = ler_registros(chave, valor)
        print("Resultados:")
        for movimento in resultado:
            print(movimento)

    elif escolha == '3':
        indice = int(input("Índice do registro a ser atualizado: "))
        while True:
            tipo = input("Tipo (Receita/Despesa/Investimento): ").capitalize()
            if tipo in ['Receita', 'Despesa', 'Investimento']:
                break
            else:
                print("Tipo inválido! Insira um tipo válido")

        valor = float(input("Valor: "))
        atualizar_registro(indice, tipo, valor)
        print("Registro atualizado com sucesso!")

    elif escolha == '4':
        indice = int(input("Índice do registro a ser deletado: "))
        deletar_registro(indice)

    elif escolha == '5':
        atualiza_rendimento()
        print("Rendimento atualizado para todos os investimentos.")

    elif escolha == '6':
        print("Seu relatório será gerado em .csv")
        exportar_relatorio()

    elif escolha == '7':
        while True:
            chave = input("Chave para agrupamento (tipo/mes/ano/data): ").lower()
            if chave in ['tipo','mes', 'ano', 'data']:
                break
            else:
                print("Chave inválida! Insira uma chave válida")
        resultado = agrupar_por(chave)
        print("Resultados:")
        print(resultado)

    elif escolha == '8':
        print("Saindo do programa. Até mais!")
        break

    else:
        print("Opção inválida. Por favor, escolha uma opção válida.")


----- Menu -----
1. Criar Registro
2. Ler Registros
3. Atualizar Registro
4. Deletar Registro
5. Atualizar Rendimento
6. Exportar Relatório
7. Agrupar Registros
8. Sair
Tipo inválido! Insira um tipo válido
Índice fora dos limites dos registros existentes, nenhum registro foi atualizado.
Registro atualizado com sucesso!

----- Menu -----
1. Criar Registro
2. Ler Registros
3. Atualizar Registro
4. Deletar Registro
5. Atualizar Rendimento
6. Exportar Relatório
7. Agrupar Registros
8. Sair
Índice fora dos limites dos registros existentes, nenhum registro foi deletado.

----- Menu -----
1. Criar Registro
2. Ler Registros
3. Atualizar Registro
4. Deletar Registro
5. Atualizar Rendimento
6. Exportar Relatório
7. Agrupar Registros
8. Sair
Rendimento atualizado para todos os investimentos.

----- Menu -----
1. Criar Registro
2. Ler Registros
3. Atualizar Registro
4. Deletar Registro
5. Atualizar Rendimento
6. Exportar Relatório
7. Agrupar Registros
8. Sair
Seu relatório será gerado em .csv
Rel